In [1]:
from typing import Text, List, Any, Dict
from rasa_sdk import Tracker
# from rasa_sdk.forms import FormAction
from rasa_sdk.forms import FormValidationAction, Action
from rasa_sdk.executor import CollectingDispatcher
from rasa_sdk.types import DomainDict
from datetime import datetime
import re


class UserDataForm(Action):
    def name(self) -> Text:
        return "user_data_form"

    @staticmethod
    def required_slots(tracker: Tracker) -> List[Text]:
        return ["name", "email", "mobile_number", "dob"]

    # def submit(
    #     self,
    #     dispatcher: CollectingDispatcher,
    #     tracker: Tracker,
    #     domain: Dict[Text, Any],
    # ) -> List[Dict[Text, Any]]:
    #     name = tracker.get_slot("name")
    #     email = tracker.get_slot("email")
    #     mobile_number = tracker.get_slot("mobile_number")
    #     dob = tracker.get_slot("dob")

    #     confirmation_message = f"Please confirm the following details:\n" \
    #                            f"Name: {name}\n" \
    #                            f"Email: {email}\n" \
    #                            f"Mobile Number: {mobile_number}\n" \
    #                            f"Date of Birth: {dob}\n" \
    #                            f"Is this information correct?"

    #     dispatcher.utter_message(text=confirmation_message)

    #     return []

class ValidateUserDataForm(FormValidationAction):
    def name(self) -> Text:
        return "validate_user_data_form"

    def validate_name(
        self,
        slot_value: Any,
        dispatcher: CollectingDispatcher,
        tracker: Tracker,
        domain: DomainDict,
    ) -> Dict[Text, Any]:
        """Validate `name` value."""
        print(f"Validating name: {slot_value}") 
        name = slot_value
        if len(name) == 0:
            dispatcher.utter_message(text="The name you entered is not correct.")
            return {"name": None}
        return {"name": name}

    def validate_mobile_number(
        self,
        slot_value: Any,
        dispatcher: CollectingDispatcher,
        tracker: Tracker,
        domain: DomainDict,
    ) -> Dict[Text, Any]:
        """Validate `mobile_number` value."""
        mobile_number = slot_value.strip() 
        if not mobile_number.isdigit() and len(mobile_number) != 11:
            dispatcher.utter_message(text="Please enter a valid 11-digit mobile number.")
            return {"mobile_number": None}

        return {"mobile_number": mobile_number}
    
    def validate_dob(
        self,
        slot_value: Any,
        dispatcher: CollectingDispatcher,
        tracker: Tracker,
        domain: DomainDict,
    ) -> Dict[Text, Any]:
        """Validate `dob` value."""
        dob = slot_value.strip()
        expected_format = "%d-%m-%Y"

        try:
            datetime.strptime(dob, expected_format)
        except ValueError:
            dispatcher.utter_message(text="Please enter a valid date of birth in the format dd-mm-yyyy.")
            return {"dob": None}

        return {"dob": dob}

    def validate_email(
        self,
        slot_value: Any,
        dispatcher: CollectingDispatcher,
        tracker: Tracker,
        domain: DomainDict,
    ) -> Dict[Text, Any]:
        """Validate `email` value."""
        email = slot_value.strip()
        email_pattern = r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$"

        if not re.match(email_pattern, email):
            dispatcher.utter_message(text="Please enter a valid email address.")
            return {"email": None}
        else:
            print(f"Email Valdation: {email}")

        return {"email": email}

class ActionConfirmData(Action):
    def name(self) -> Text:
        return "action_confirm_data"

    def run(
        self, dispatcher: CollectingDispatcher, tracker: Tracker, domain: Dict[Text, Any]
    ) -> List[Dict[Text, Any]]:
        user_input = tracker.latest_message.get("text")
        affirm_intent = "affirm_data"  
        deny_intent = "deny_data"      

        if affirm_intent in tracker.latest_message['intent']['name']:
            response = domain['responses']['affirmation_response']
        elif deny_intent in tracker.latest_message['intent']['name']:
            response = domain['responses']['denial_response']
        else:
            response = domain['responses']['invalid_response']

        dispatcher.utter_message(template=response)
        return []






ModuleNotFoundError: No module named 'rasa_sdk'

In [3]:
import psycopg2

db_connection = {
    "user": "postgres",
    "password": "1947001",
    "host": "localhost",
    "port": "5432",
    "database": "Form"
}

def create_table():
    try:
        conn = psycopg2.connect(**db_connection)
        cur = conn.cursor()
        cur.execute('''
            CREATE TABLE IF NOT EXISTS user_data (
                id serial PRIMARY KEY,
                name VARCHAR(255),
                phone_number VARCHAR(11),
                email VARCHAR(255),
                dob DATE
            )
        ''')
        conn.commit()
    except Exception as e:
        print("Error creating table:", e)
    finally:
        cur.close()
        conn.close()

def save_user_data(name, phone_number, email, dob):
    try:
        conn = psycopg2.connect(**db_connection)
        cur = conn.cursor()

        cur.execute("INSERT INTO user_data (name, phone_number, email, dob) VALUES (%s, %s, %s, %s)",
                    (name, phone_number, email, dob))
        conn.commit()
    except Exception as e:
        print("Error saving form data:", e)
    finally:
        conn.commit()
        cur.close()
        conn.close()

def save_to_database(name, phone_number, email, dob):
    try:
        conn = psycopg2.connect(**db_connection)
        cur = conn.cursor()

        cur.execute("INSERT INTO form_data (name, phone_number, email, dob) VALUES (%s, %s, %s, %s)", (name, phone_number, email, dob))
        conn.commit()
    except Exception as e:
        print("Error saving data to the database:", e)
    finally:
        cur.close()
        conn.close()

In [4]:
class SubmitUserData(Action):

      def name(self) -> Text:
        return "action_submit_data"

      def submit(
        self,
        dispatcher: CollectingDispatcher,
        tracker: Tracker,
        domain: Dict[Text, Any],
    ) -> List[Dict[Text, Any]]:
        name = tracker.get_slot("name")
        email = tracker.get_slot("email")
        mobile_number = tracker.get_slot("mobile_number")
        dob = tracker.get_slot("dob")

        confirmation_message = f"Please confirm the following details:\n" \
                               f"Name: {name}\n" \
                               f"Email: {email}\n" \
                               f"Mobile Number: {mobile_number}\n" \
                               f"Date of Birth: {dob}\n" \
                               f"Is this information correct?"

        dispatcher.utter_message(text=confirmation_message)

        return []




class ActionHandleAffirmation(Action):
    def name(self) -> Text:
        return "action_handle_affirmation"

    def run(self, dispatcher: CollectingDispatcher, tracker: Tracker, domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:
       
        dispatcher.utter_message("The data you entered is confirmed.")
        return []

class ActionHandleDenial(Action):
    def name(self) -> Text:
        return "action_handle_denial"

    def run(self, dispatcher: CollectingDispatcher, tracker: Tracker, domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:
        dispatcher.utter_message("Can you provide more information?")
        return []


class ActionSubmitData(Action):
    def name(self) -> Text:
        return "action_submit_data"

    def run(self, dispatcher: CollectingDispatcher, tracker: Tracker, domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:
        name = tracker.get_slot("name") 
        phone = tracker.get_slot("phone")
        email = tracker.get_slot("email")
        dob = tracker.get_slot("dob")

        save_to_database(name, phone, email, dob)
        dispatcher.utter_message("Your data has been submitted.")

        return []





NameError: name 'Action' is not defined

In [9]:
import psycopg2

db_connection = {
    "user": "postgres",
    "password": "1947001",
    "host": "localhost",
    "port": "5433",
    "database": "Form"
}

In [10]:
conn = psycopg2.connect(**db_connection)
cur = conn.cursor()
cur.execute('''
            CREATE TABLE IF NOT EXISTS user_data (
                id serial PRIMARY KEY,
                name VARCHAR(255),
                phone_number VARCHAR(11),
                email VARCHAR(255),
                dob DATE
            )
        ''')

In [22]:
conn = psycopg2.connect(**db_connection)
cur = conn.cursor()

cur.execute("INSERT INTO user_data (name, phone, email, dob) VALUES ('Emma', '10293847561', 'emma123@qwerty.com', '10-10-2010')")
conn.commit()
 